# Implement Multi Agent Framework

In [8]:
!pip install python-dotenv --quiet
!pip install tenacity --quiet
!pip install openai --quiet

In [2]:
from openai import OpenAI
from typing import Optional
import os
from dotenv import load_dotenv
from tenacity import retry, wait_random_exponential, stop_after_attempt
from typing import List, Optional, Dict
import logging

### LLM 预备

In [3]:
# Load environment variables from .env file
load_dotenv()

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url=os.environ.get("OPENAI_BASE_URL"),
)

### Agent & Multi-Agent 构建

In [4]:

class BaseAgent:
    """
    An example of a base agent class, including an OpenAI client.
    """
    def __init__(self,client:OpenAI):
        self.client = client
        # can add more attributes and methods here
        

class Agent:
    """
    A class representing an agent that can handle chat messages and handoff to other agents.
    """
    def __init__(
            self,
            agent:BaseAgent,
            name:str,
            instructions: Optional[str] = None):
        self.DEFAULT_MODEL = "gpt-4o-mini"
        self.name = name.replace(" ","_")
        self.agent = agent
        self.instructions = instructions
        self.handoff_agent_schemas: List[Dict] = []
        self.handoff_agents: Dict[str, "Agent"] = {}

    def __str__(self):
        """
        Returns a string representation of the Agent instance.
        """
        return f"Agent(name={self.name}, instructions={self.instructions},handoff_agents={list(self.handoff_agents.keys())})"


    def add_handoffs(self, agents: List["Agent"]) -> None:
        """ 
        Adds handoff agents to the current agent.
        """
        self.handoff_agent_schemas.clear()
        for agent in agents:
            agent_schema = {
                'type': 'function',
                'function': {
                    'name': agent.name,
                    'description': agent.instructions or '',
                    'parameters': {'type': 'object', 'properties': {}, 'required': []}
                }
            }
            self.handoff_agent_schemas.append(agent_schema)
            self.handoff_agents[agent.name] = agent

    @retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
    def chat(self,messages:List[Dict],model: Optional[str] = None,disable_tools:bool = False):
        """ 
        Handles chat messages and returns responses, with optional handoff to other agents.
        """

        if self.instructions:
            messages = [{"role": "system", "content": self.instructions}] + messages
        
        model = model or self.DEFAULT_MODEL

        try:

            response = self.agent.client.chat.completions.create(
                model=model,
                messages=messages,
                tools=None if disable_tools else self.handoff_agent_schemas or None,
                tool_choice=None,
            )
            message = response.choices[0].message
            if not message.tool_calls:
                result = [{"role": "assistant", "content": message.content, "agent_name": self.name}]
                return result
            
            result = []
            for tool_call in message.tool_calls:
                call_agent = self.handoff_agents[tool_call.function.name]
                call_message = {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "handoff": f"{self.name} -> {call_agent.name}",
                    "agent_name": call_agent.name,
                    "agent": call_agent,
                }
                result.append(call_message)      
            return result
        except Exception as e:
            logging.error("Unable to generate ChatCompletion response")
            logging.error(f"Exception: {e}")
            return None


class MultiAgent:
    def __init__(self,start_agent:Optional[Agent] = None):
        """
        Initialize the MultiAgent with an optional starting agent.
        """
        self.current_agent = start_agent

    def add_handoff_relations(self,from_agent:Agent,to_agents:List[Agent])->None:
        """
        Add handoff relations from one agent to multiple agents.
        """
        from_agent.add_handoffs(to_agents)

    def chat(self,messages:List[Dict],model: Optional[str] = None,agent:Optional[Agent] = None,max_handoof_depth:int = 2)->List[Dict]:
        """
        Chat with the current agent or a specified agent using the provided messages and model.
        """
        if agent:
            self.current_agent = agent

        if not self.current_agent:
            logging.error("No agent is set for conversation. You can set it in init or pass as argument.")
            return None
        
        try:
            # currently only one handoff is supported
            res = [self.current_agent.chat(messages, model)[0]]
            if res and res[0].get("content"):
                return res
            while res and res[-1].get("tool_call_id") and max_handoof_depth > 0:
                max_handoof_depth -= 1
                self.current_agent = res[-1]["agent"]
                if max_handoof_depth == 0:
                    res.extend(self.current_agent.chat(messages, model, disable_tools=True))
                else:
                    res.extend(self.current_agent.chat(messages, model))
            return res
        except Exception as e:
            logging.error("Error during chat with agent.")
            logging.error(f"Exception: {e}")
            return None
        
    def handoff(self,messages:List[Dict],model: Optional[str] = None,agent:Agent = None)-> Agent:
        """
        Perform a handoff to the specified agent using the provided messages and model.
        """
        current_agent = agent

        if not current_agent:
            logging.error("No agent is set for conversation. You must specify an agent.")
            return None
        
        try:
            res = current_agent.chat(messages, model)
            handoff_agents = {}
            if res:
                for r in res:
                    if r.get("tool_call_id"):
                        handoff_agents[r["agent_name"]] = r["agent"]
            if handoff_agents:
                return handoff_agents
            return None
        except Exception as e:
            logging.error("Error during handoff with agent.")
            logging.error(f"Exception: {e}")
            return None

### 初始化几个Agent

In [5]:
a1 = Agent(BaseAgent(client),"general agent","Agent used for daily questions named Ada.")
a2 = Agent(BaseAgent(client),"science agent","Agent used for science questions named Albert.")
a3 = Agent(BaseAgent(client),"music agent","Agent used for music questions named Mozart.")

### 创建Multi-Agent环境 & 增加handoff的关系

In [6]:
ma = MultiAgent(a1)
ma.add_handoff_relations(a1,[a2,a3])
ma.add_handoff_relations(a2,[a1])
ma.add_handoff_relations(a3,[a1])
# ma.add_handoff_relations(a2,[a1,a3])
# ma.add_handoff_relations(a3,[a1,a2])

### Multi-Agent handoff 的能力展示

调用handoff得到合适完成任务的agent

随后调用这个agent预先设计好的各种能力，比如 streaming, tools, memory ,reasoning 等等

In [7]:
ma.handoff(messages=[{"role": "user", "content": "why the sky is blue"}],agent=a1)

{'science_agent': <__main__.Agent at 0x114dd7d90>}

In [8]:
ma.handoff(messages=[{"role": "user", "content": "why the sky is blue and recommend me some music"}],agent=a1)

{'science_agent': <__main__.Agent at 0x114dd7d90>,
 'music_agent': <__main__.Agent at 0x114dd6950>}

### Multi-Agent 对话能力展示 

自带handoff的agent, 可以直接进行对话,也可自动将对话交接给其他合适的agent进行回复

In [9]:
ma.chat(messages=[{"role": "user", "content": "why the sky is blue"}])

[{'role': 'tool',
  'tool_call_id': 'call_457DNbiYWTudtC3n0PMmpwsL',
  'handoff': 'general_agent -> science_agent',
  'agent_name': 'science_agent',
  'agent': <__main__.Agent at 0x114dd7d90>},
 {'role': 'assistant',
  'content': "The sky appears blue due to a phenomenon called Rayleigh scattering. This occurs when sunlight passes through the Earth's atmosphere and interacts with the gas molecules and small particles in the air.\n\nHere’s how it works:\n\n1. **Sunlight Composition**: Sunlight, or white light, is made up of different colors, each with different wavelengths. Blue light has a shorter wavelength compared to other colors like red.\n\n2. **Scattering of Light**: When sunlight enters the atmosphere, the shorter blue wavelengths are scattered in all directions by the oxygen and nitrogen molecules in the air. \n\n3. **Perception of Color**: Since blue light is scattered more than other colors, when we look up at the sky, we see a higher concentration of blue light compared to o

In [10]:
ma.chat(messages=[{"role": "user", "content": "who are you"}])

[{'role': 'assistant',
  'content': 'I am Albert, an AI trained to assist with science-related questions. How can I help you today?',
  'agent_name': 'science_agent'}]

In [11]:
ma.chat(messages=[{"role": "user", "content": "What are the different music styles?"}])

[{'role': 'tool',
  'tool_call_id': 'call_vM7dNjb1xOZB3bEVTIHW7QLD',
  'handoff': 'science_agent -> general_agent',
  'agent_name': 'general_agent',
  'agent': <__main__.Agent at 0x114dd7ad0>},
 {'role': 'tool',
  'tool_call_id': 'call_SwYrjE0bg1FoIRMwnDfxEd21',
  'handoff': 'general_agent -> music_agent',
  'agent_name': 'music_agent',
  'agent': <__main__.Agent at 0x114dd6950>},
 {'role': 'assistant',
  'content': 'Music encompasses a vast array of styles, each characterized by unique elements such as rhythm, melody, instrumentation, and cultural influences. Here are some of the major music styles:\n\n1. **Classical**: Encompasses various periods (Baroque, Classical, Romantic, etc.) and is characterized by orchestral arrangements, complex compositions, and a focus on melody and harmony.\n\n2. **Jazz**: Originating in the African-American communities, jazz is known for its improvisation, swing rhythms, and use of blues influences. Subgenres include bebop, smooth jazz, and jazz fusion.

In [12]:
ma.chat(messages=[{"role": "user", "content": "dinner tonight"}])

[{'role': 'tool',
  'tool_call_id': 'call_ur9N9ky1ATwZE67socCzIwhJ',
  'handoff': 'music_agent -> general_agent',
  'agent_name': 'general_agent',
  'agent': <__main__.Agent at 0x114dd7ad0>},
 {'role': 'assistant',
  'content': 'What kind of dinner are you in the mood for tonight? Are you looking for a specific cuisine, or do you have any dietary preferences or restrictions? Let me know so I can help you decide!',
  'agent_name': 'general_agent'}]

In [13]:
ma.chat(messages=[{"role": "user", "content": "who are you"}],agent=a2)

[{'role': 'assistant',
  'content': 'I am Albert, your assistant for science-related questions. How can I help you today?',
  'agent_name': 'science_agent'}]